In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

import os
import sys
import glob
sys.path.insert(1, '/home/cem/Documents/implicit_scene/LLFF')
sys.path.insert(1, '/home/cem/Documents/implicit_scene/nerf_pl')
sys.path.insert(1, '/home/cem/Desktop/ov-workspace/src')

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import tensorflow.keras.layers as tfkl

from imps.nerf.data_utils import load_colmap_poses_bounds, load_ov_poses_bounds, generate_data
from imps.nerf.model import get_embedder, get_single_nerf
from imps.nerf.ray_utils import sample_points_on_ray
from imps.vis import CameraPoseVisualizer

from imps.nerf.render import render_hierarchical, render_single_model

DATA_DIR = '/home/cem/Desktop/ov-workspace/data/ov-v3'
OUT_DIR = os.path.join(DATA_DIR, 'train')
IMAGE_EXT = '.png'

N_coarse = 64
N_fine = 128

Lp = 10
Ld = 4

lr = 5e-4

batch_size = 4096
num_epoch = 10
embedding_include_input = True

2021-09-21 21:05:49.756591: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
image_files = sorted(glob.glob(os.path.join(DATA_DIR, f'images/*{IMAGE_EXT}')))
poses_bounds = load_colmap_poses_bounds(DATA_DIR)
origins, viewdirs, imgs, bounds = generate_data(image_files, poses_bounds, down_scale=8)

Cameras 5
Images # 96
Points (26017, 3) Visibility (26017, 96)
Depth stats 0.11425793061809168 105.8574979259575 5.746784778375583


### Ray shooting test

In [ ]:
# Test the generated rays
cp_vis = CameraPoseVisualizer([-3, 3], [-3, 3], [0, 3])

for i in range(len(image_files)):
    ray_o = origins[i]
    ray_dir = viewdirs[i]
    
    cam_params = poses_bounds[i, :-2].reshape(3, 5)
    cam_extrinsics = cam_params[:, :-1]

    p0 = np.concatenate((cam_extrinsics, np.array([[0,0,0,1]])), axis=0)
    pix_loc = ray_o + ray_dir

    cp_vis.extrinsic2pyramid(p0, focal_len_scaled=0.25, aspect_ratio=0.56)

    for j in range(0, 1280, 100):
        cp_vis.ax.plot3D([ray_o[360, j, 0], pix_loc[360, j, 0]],
                         [ray_o[360, j, 1], pix_loc[360, j, 1]],
                         [ray_o[360, j, 2], pix_loc[360, j, 2]], ls=':', color='black')   